In [0]:
import numpy as np
import os 
import cv2 as cv
import keras
import matplotlib.pyplot as plt
import gzip
import idx2numpy
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from keras import models , regularizers, layers, losses
from random import shuffle
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import load_img,img_to_array

np.random.seed(seed=0) #Constant Seed at 0

def shuffleData(Data,Labels): #This is for shuffeling data
    m=Data.shape[0]
    permutation=list(np.random.permutation(m))
    Data=Data[permutation,:,:]
    Labels=Labels[permutation]
    return Data,Labels
  
def PlotModelGraph(Model_History): #This is for plotting the model after its done
    print(Model_History.history.keys())
    #Plot the accuracy
    plt.plot(Model_History.history['acc'])
    plt.plot(Model_History.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()
    #Plot the loss
    plt.plot(Model_History.history['loss'])
    plt.plot(Model_History.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()
    return
  
  
Training_Data_Directory = './train-images.idx3-ubyte'
Training_Data = idx2numpy.convert_from_file(Training_Data_Directory) #Reading Data
Training_Data = Training_Data.reshape(60000, 28, 28, 1)  #Making sure shapes are correct

Training_Labels_Directory = './train-labels.idx1-ubyte'
Training_Labels = idx2numpy.convert_from_file(Training_Labels_Directory) #Reading Data

Test_Data_Directory = './t10k-images.idx3-ubyte'
Test_Data = idx2numpy.convert_from_file(Test_Data_Directory) #Reading Data
Test_Data = Test_Data.reshape(10000, 28, 28, 1)  #Making sure shapes are correct

Test_Labels_Directory = './t10k-labels.idx1-ubyte'
Test_Labels = idx2numpy.convert_from_file(Test_Labels_Directory) #Reading Data


  
Shuffled_Training_Data, Shuffled_Training_Labels = shuffleData(Training_Data, Training_Labels)  #Shuffeling the data
Shuffled_Test_Data, Shuffled_Test_Labels = shuffleData(Test_Data, Test_Labels)  #Shuffeling the data


Training_Labels = to_categorical(Shuffled_Training_Labels)  #Creating 1 hot vector for the labels
Test_Labels = to_categorical(Shuffled_Test_Labels)  #Creating 1 hot vector for the labels

#---------------------------Model Architecture---------------------------------
model = Sequential()
model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=5, activation='relu'))
model.add(Conv2D(32, kernel_size=5, activation='relu'))
model.add(Conv2D(32, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
#------------------------------------------------------------------------------
#Optimizers i used and experimented with
Adam_optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)  
SGD = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

model.compile(optimizer = Adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])  #Compiling the model

History = model.fit(Shuffled_Training_Data, Training_Labels, validation_data = (Shuffled_Test_Data, Test_Labels), batch_size= 200, epochs=10)  #Training

PlotModelGraph(History)   #Plotting 


Predicted_Classes = model.predict_classes(Shuffled_Test_Data, verbose=0)   #Predicting results for the test on the trained model

precision, recall, fscore, support = score(Shuffled_Test_Labels, Predicted_Classes)    #Getting scores

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))